<a href="https://colab.research.google.com/github/siripragadashashank/sagemaker-huggingface-llama-2-samples/blob/master/LLama2-7b_chunk_vs_performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/vector_stores/SimpleIndexDemoLlama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama2 + VectorStoreIndex

This notebook walks through the proper setup to use llama-2 with LlamaIndex. Specifically, we look at the impact of chunk size on the performance metrics of LLama-2-7b.

## Setup

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

### Keys

In [19]:
!pip install openai
!pip install llama_index
!pip install langchain
!pip install sentence_transformers
!pip install pypdf
!pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 4.4 MB/s eta 0:00:00


In [20]:
import os

# Please obtain OpenAI and Replicate API keys and paste them here.
os.environ["OPENAI_API_KEY"] = "*****"
os.environ["REPLICATE_API_TOKEN"] = "*****"

# currently needed for notebooks
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

### Load documents, build the VectorStoreIndex

In [21]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
)

from IPython.display import Markdown, display

In [22]:
from llama_index.llms import Replicate
from llama_index import ServiceContext, set_global_service_context
from llama_index.llms.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

from langchain.embeddings import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# The replicate endpoint
LLAMA_7B_V2_CHAT = "meta/llama-2-7b-chat:13c3cdee13ee059ab779f0291d29054dab00a47dad8261375654de5540165fb0"


# inject custom system prompt into llama-2
def custom_completion_to_prompt(completion: str) -> str:
    return completion_to_prompt(
        completion,
        system_prompt=(
            "You are a Q&A assistant. Your goal is to answer questions as "
            "accurately as possible is the instructions and context provided."
        ),
    )


llm = Replicate(
    model=LLAMA_7B_V2_CHAT,
    temperature=0.01,
    # override max tokens since it's interpreted
    # as context window instead of max tokens
    context_window=4096,
    # override completion representation for llama 2
    completion_to_prompt=custom_completion_to_prompt,
    # if using llama 2 for data agents, also override the message representation
    messages_to_prompt=messages_to_prompt,
)

# set a global service context
ctx = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)
set_global_service_context(ctx)

Download Data

In [23]:
import nest_asyncio
import time
nest_asyncio.apply()

from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.evaluation import (
    DatasetGenerator,
    FaithfulnessEvaluator,
    RelevancyEvaluator
)

In [24]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/jerryjliu/llama_index/main/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'

--2023-12-17 06:35:22--  https://raw.githubusercontent.com/jerryjliu/llama_index/main/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.06s   

2023-12-17 06:35:22 (30.9 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]



In [26]:
reader = SimpleDirectoryReader("./data/10k/")
documents = reader.load_data()

# To evaluate for each chunk size, we will first generate a set of 40 questions from first 20 pages.
eval_documents = documents[:40]
data_generator = DatasetGenerator.from_documents(documents=eval_documents)
eval_questions = data_generator.generate_questions_from_nodes(num = 40)


service_context_llama2 = ServiceContext.from_defaults(embed_model=embed_model,
                                                      llm=llm)

# Define Faithfulness and Relevancy Evaluators which are based on LLama-2
faithfulness_llama2 = FaithfulnessEvaluator(service_context=service_context_llama2)
relevancy_llama2 = RelevancyEvaluator(service_context=service_context_llama2)
def evaluate_response_time_and_accuracy(chunk_size):
    total_response_time = 0
    total_faithfulness = 0
    total_relevancy = 0

    # create vector index
    service_context = ServiceContext.from_defaults(
        embed_model=embed_model,
        llm=llm,
        chunk_size=chunk_size
        )

    vector_index = VectorStoreIndex.from_documents(
        eval_documents, service_context=service_context
    )

    query_engine = vector_index.as_query_engine()
    num_questions = len(eval_questions)

    for question in eval_questions:
        start_time = time.time()
        response_vector = query_engine.query(question)
        elapsed_time = time.time() - start_time

        faithfulness_result = faithfulness_llama2.evaluate_response(
            response=response_vector
        ).passing

        relevancy_result = relevancy_llama2.evaluate_response(
            query=question, response=response_vector
        ).passing

        total_response_time += elapsed_time
        total_faithfulness += faithfulness_result
        total_relevancy += relevancy_result

    average_response_time = total_response_time / num_questions
    average_faithfulness = total_faithfulness / num_questions
    average_relevancy = total_relevancy / num_questions

    return average_response_time, average_faithfulness, average_relevancy

# Iterate over different chunk sizes to evaluate the metrics to help fix the chunk size.
for chunk_size in [128, 256, 512, 1024, 2048]:
  avg_time, avg_faithfulness, avg_relevancy = evaluate_response_time_and_accuracy(chunk_size)
  print(f"Chunk size {chunk_size} - Average Response time: {avg_time:.2f}s, Average Faithfulness: {avg_faithfulness:.2f}, Average Relevancy: {avg_relevancy:.2f}")

chunk_size_limit is deprecated, please specify chunk_size instead
chunk_size_limit is deprecated, please specify chunk_size instead
chunk_size_limit is deprecated, please specify chunk_size instead


/usr/local/lib/python3.10/dist-packages/llama_index/evaluation/dataset_generation.py:187: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(
/usr/local/lib/python3.10/dist-packages/llama_index/evaluation/dataset_generation.py:282: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


Chunk size 128 - Average Response time: 1.87s, Average Faithfulness: 1.00, Average Relevancy: 0.95
Chunk size 256 - Average Response time: 1.89s, Average Faithfulness: 1.00, Average Relevancy: 0.95
Chunk size 512 - Average Response time: 1.89s, Average Faithfulness: 1.00, Average Relevancy: 0.95
Chunk size 1024 - Average Response time: 1.89s, Average Faithfulness: 1.00, Average Relevancy: 0.95
Chunk size 2048 - Average Response time: 1.96s, Average Faithfulness: 1.00, Average Relevancy: 0.95
